# **Pandas - pivot & pivot_table**

피봇테이블(pivot table)이란, 데이터에서 두 개의 컬럼을 행/열 인덱스로 써서 데이터를 reshape한 테이블을 말한다. 기존의 데이터를 새로운 기준으로 집계해서 정리할 수 있는 방법이라고 이해하면 좋을 것 같다! 아래에서 실제 피봇테이블의 결과물을 보면 무슨 말인지 쉽게 이해가 될 수 있을 것이다. <br>
여기에서는 간단한 실제 데이터를 이용해서 pandas에서 제공하는 `pivot`과 `pivot_table` 메서드를 통해 데이터를 새롭게 집계하는 방법을 알아보자.

## **1. Pivot**
- 데이터 프레임의 컬럼 데이터에서 index, columns, values를 선택하여 데이터 프레임을 만드는 방법
- syntax: `df.pivot(index, columns, values)`
- 유의할 점: 지정한 index와 columns를 기준으로 선택되는 데이터(values 파라미터에 해당)가 유일해야 한다! values가 2개 이상이면 에러 발생 <br> → 따라서 pivot을 하기 전에 index와 column으로 사용할 컬럼을 기준으로 groupby를 먼저 해주어야 한다.

무엇이든 실제 사용 예를 보아야 이해가 쉽다! kaggle에서 제공하는 타이타닉 데이터를 사용해 간단하게 pivot을 실습해보자.
- 타이타닉 데이터: 타이타닉 승객 정보를 담은 데이터로, 생존여부를 예측하는 문제가 kaggle에 올라와 있음
- 예측 대상인 생존여부는 Survived 컬럼 (생존=1, 사망=0) 
- [데이터 다운로드](https://www.kaggle.com/c/titanic/data)
    - 아래에서는 제공되는 데이터셋 중에서 train.csv만 사용한다.

In [1]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

In [2]:
titanic = pd.read_csv("titanic_train.csv")
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### 1.1 성별과 객실등급에 따라 승객 수 집계하기

위의 titanic 데이터 프레임에서 "Sex"와 "Pclass"(객실등급)를 index와 columns으로 삼아 pivot을 해보도록 하자. <br>
우선, 기존의 데이터프레임 중 두 개의 컬럼 "Sex"와 "Pclass"으로 구성된 새로운 데이터프레임을 만들어보자.

In [3]:
titanic_f1 = pd.DataFrame(titanic, columns=["Sex", "Pclass"])
titanic_f1.tail()

,Sex,Pclass
886,male,2
887,female,1
888,female,3
889,male,1
890,male,3


titanic_f1 데이터프레임에는 특정 "Pclass" & 특정 "Sex" 조합에 해당하는 데이터가 여러 번 나타날 것이므로, 바로 pivot 메서드를 사용하게되면 에러가 발생하기 때문에  먼저 groupby를 해주어야 한다. "Sex"와 "Pclass"로 groupby를 해서 데이터의 수를 "Counts" 칼럼으로 만들어주자. <br>
성별은 female & male, 객실등급은 1,2,3 등급이 있으므로 6가지 조합의 데이터 수를 결과로 얻게될 것이다.

In [4]:
titanic_f1 = titanic.groupby(["Sex","Pclass"]).size().reset_index(name="Counts")
titanic_f1

,Sex,Pclass,Counts
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


이제 pivot 메서드를 통해 "Sex"를 index로, "Pclass"를 columns로 하고 "Counts"가 values인 새로운 데이터프레임을 만들 수 있다.

In [5]:
titanic_f1.pivot("Sex","Pclass","Counts")

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


### 1.2 성별과 생존여부에 따라 승객 수 집계하기

위와 같은 방법을 적용하면 된다. 먼저 전체 데이터프레임에서 "Sex"와 "Survived"(생존여부) 컬럼을 기준으로 데이터 수를 groupby 해준다. <br>(위에서는 2개의 컬럼으로 우선 별도의 데이터프레임을 만들고 나서 groupby를 진행했지만 바로 groupby를 해도 결과는 동일하다.)

In [6]:
titanic_f2 = titanic.groupby(["Sex","Survived"]).size().reset_index(name='Counts')
titanic_f2

,Sex,Survived,Counts
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109


groupby된 데이터프레임에 대해 "Sex"를 index로, "Survived"를 columns로 두고 "Counts"가 values가 되도록 pivot한다.

In [7]:
titanic_f2.pivot("Sex","Survived","Counts")

Survived,0,1
Sex,,
female,81,233
male,468,109


## **2. Pivot table**
pivot_table 메서드를 이용하면 한번에 groupby를 한 후에 pivot을 하는 것과 같은 결과를 얻을 수 있다.
- syntax: `pivot_table(values, index, columns, aggfunc='mean', fill_value=None, dropna=True, margins=False, margins_name='All')`
    - `pivot` 메서드와 달리 `values` 파라미터가 먼저 들어간다는 점에 유의!
    - `aggfunc`: index와 columns로 선택되는 데이터가 유일하지 않으면 인수로 넣어주는 함수를 수행하여 대표값을 계산함
    - `fill_value`: NaN을 대체할 값
    - `dropna`: values가 전부 NaN인 행/열의 삭제 여부
    - `margins`: 모든 데이터를 분석한 결과(=마진)를 마지막 행/열에 붙일지 여부
    - `margins_name`: 마진 행/열의 이름


### 2.1 성별과 객실등급에 따라 승객 수 집계하기

`pivot_table` 메서드를 사용해 `groupby` → `pivot`을 사용했을 때와 동일한 결과를 만들어보자. <br> 
우선 조건(index, columns)별 데이터 수를 카운트하는데 사용할 목적으로 "Counts" 컬럼을  만들어 준 후에 `pivot_table` 메서드를 사용한다.

In [8]:
titanic["Counts"] = 1
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Counts
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,1
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,1


In [9]:
titanic.pivot_table("Counts",["Sex"],["Pclass"], aggfunc = np.sum)

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


### 2.2 성별과 생존여부에 따라 승객 수 집계하기

In [10]:
titanic.pivot_table("Counts",["Sex"],["Survived"], aggfunc = np.sum)
# titanic.pivot_table("Counts","Sex","Survived", aggfunc = np.sum)

Survived,0,1
Sex,,
female,81,233
male,468,109


### 2.3 다중 index, 다중 columns 사용하기
`pivot_table`의 index나 columns 파라미터에 list를 넣어주면 multi-level index/column을 만들 수 있다.

In [11]:
# 2-level index
titanic.pivot_table("Counts",["Pclass","Sex"],["Survived"], aggfunc = np.sum)

Survived         0   1
Pclass Sex            
1      female    3  91
       male     77  45
2      female    6  70
       male     91  17
3      female   72  72
       male    300  47

In [12]:
# 2-level columns
titanic.pivot_table("Counts",["Survived"], ["Pclass","Sex"], aggfunc = np.sum)

Pclass        1           2           3     
Sex      female male female male female male
Survived                                    
0             3   77      6   91     72  300
1            91   45     70   17     72   47

### 2.4 pivot table의 행/열을 집계하는 Margin 컬럼 만들기
각 행/열의 총 합계를 집계하는 margin 행/열을 만들 수 있다. `pivot_table` 메서드의 관련 파라미터를 사용할 수도 있고, 직접 계산해서 행/열을 추가해줄 수도 있다.

#### (1) `pivot_table` 메서드의 `margins` 파라미터 사용

- `margins` 파라미터를 True로 넣어주면 margin 행과 열이 생성된다.
- `margins_name` 파라미터로 margin 행/열의 이름을 지정할 수 있다. (기본값="All")

In [28]:
titanic.pivot_table("Counts",["Survived"], ["Sex"], aggfunc = np.sum, margins=True, margins_name="Total")

Sex,female,male,Total
Survived,,,
0,81,468,549
1,233,109,342
Total,314,577,891


#### (2) 직접 margin 행/열 추가

In [24]:
# total column 추가
survived_df = titanic.pivot_table("Counts",["Survived"],["Sex"], aggfunc = np.sum)
survived_df["Total"] = survived_df["female"] + survived_df["male"]
survived_df

Sex,female,male,Total
Survived,,,
0,81,468,549
1,233,109,342


In [25]:
# total row 추가
survived_df.loc["Total"] = survived_df.loc[0] + survived_df.loc[1]
survived_df

Sex,female,male,Total
Survived,,,
0,81,468,549
1,233,109,342
Total,314,577,891


### 2.5 결측값의 처리 - fill_value / dropna

#### (1) fill_value
- 데이터가 없을 때(=NaN) 대체할 값을 지정하는 파라미터

In [29]:
titanic.pivot_table("Counts", "Survived", ["Parch","Pclass"],
                    aggfunc = np.sum)

Parch        0                  1                 2                3         4         5    6
Pclass       1     2      3     1     2     3     1     2     3    2    3    1    3    3    3
Survived                                                                                     
0         64.0  86.0  295.0  10.0   8.0  35.0   5.0   3.0  32.0  NaN  2.0  1.0  3.0  4.0  1.0
1         99.0  48.0   86.0  21.0  24.0  20.0  16.0  13.0  11.0  2.0  1.0  NaN  NaN  1.0  NaN

In [30]:
titanic.pivot_table("Counts", "Survived", ["Parch","Pclass"],
                    aggfunc = np.sum, fill_value = 0)

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

#### (2) dropna
- dropna 파라미터에 False를 넣어주면 집계 결과가 없는 데이터의 행/열까지 모두 표시함 (default는 True)

In [31]:
# True(default): 데이터가 없는(NaN) 행/열은 삭제됨
titanic.pivot_table("Counts", "Survived", ["Parch","Pclass"],
                    aggfunc = np.sum, fill_value = 0)

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

In [32]:
# False: 데이터가 없는(NaN) 행/열도 표시함 (가능한 조합이 전부 표시됨)
titanic.pivot_table("Counts", "Survived", ["Parch","Pclass"],
                    aggfunc = np.sum, fill_value = 0, dropna = False)

Parch      0            1           2          3        4        5        6      
Pclass     1   2    3   1   2   3   1   2   3  1  2  3  1  2  3  1  2  3  1  2  3
Survived                                                                         
0         64  86  295  10   8  35   5   3  32  0  0  2  1  0  3  0  0  4  0  0  1
1         99  48   86  21  24  20  16  13  11  0  2  1  0  0  0  0  0  1  0  0  0

#### 참고자료
- 패스트캠퍼스, ⟪데이터사이언스스쿨 8기⟫ 수업자료
- pandas 0.25.0 documentation 
    - [pandas.DataFrame.pivot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html)
    - [pandas.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)